In [1]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import sys
from datetime import datetime
import time
%matplotlib inline

### Organize the original data ###

In [2]:
main_data = pd.read_csv("data_with_station_cities.csv", delimiter=",", low_memory=False)

print "Dimension of the data: ", main_data.shape
main_data.head()

Dimension of the data:  (4586616, 20)


,Unnamed: 0,tripduration,starttime,stoptime,start.station.id,start.station.name,start.station.latitude,start.station.longitude,end.station.id,end.station.name,end.station.latitude,end.station.longitude,bikeid,usertype,birth.year,gender,start_neighborhood,start_borough,end_neighborhood,end_borough
0,0,191,7/1/2016 00:00:19,7/1/2016 00:03:31,3090,N 8 St & Driggs Ave,40.71775,-73.95600,3107,Bedford Ave & Nassau Ave,40.72312,-73.95212,16345,Subscriber,1986.0,2,Greenpoint,Brooklyn,Greenpoint,Brooklyn
1,1,687,7/1/2016 00:00:24,7/1/2016 00:11:52,459,W 20 St & 11 Ave,40.74674,-74.00776,347,Greenwich St & W Houston St,40.72885,-74.00859,25210,Customer,NaN,0,Chelsea and Clinton,Manhattan,Greenwich Village and Soho,Manhattan
2,2,609,7/1/2016 00:00:31,7/1/2016 00:10:40,284,Greenwich Ave & 8 Ave,40.73902,-74.00264,212,W 16 St & The High Line,40.74335,-74.00682,15514,Customer,NaN,0,Greenwich Village and Soho,Manhattan,Chelsea and Clinton,Manhattan
3,3,672,7/1/2016 00:00:45,7/1/2016 00:11:57,459,W 20 St & 11 Ave,40.74674,-74.00776,347,Greenwich St & W Houston St,40.72885,-74.00859,15444,Subscriber,1991.0,1,Chelsea and Clinton,Manhattan,Greenwich Village and Soho,Manhattan
4,4,466,7/1/2016 00:00:49,7/1/2016 00:08:35,128,MacDougal St & Prince St,40.72710,-74.00297,250,Lafayette St & Jersey St N,40.72456,-73.99565,14676,Subscriber,1989.0,1,Greenwich Village and Soho,Manhattan,Greenwich Village and Soho,Manhattan


In [6]:
# number of trips
n =  main_data.shape[0]

# extract the time columns
start_time = main_data['starttime']
stop_time = main_data['stoptime']
# store the results
month = []
day_of_month = []
day_of_week = []
start_hour = []
stop_hour = []

# for each trip
for i in range(n):
    # separate start time into month, day, start_hour
    trip_start_time = time.strptime(start_time[i], "%m/%d/%Y %H:%M:%S")
    month.append(trip_start_time.tm_mon)
    day_of_month.append(trip_start_time.tm_mday)
    start_hour.append(trip_start_time.tm_hour)
    # extract weekday
    day_of_week.append(trip_start_time.tm_wday)

    # extract stop_hour from stop_time
    trip_stop_time = time.strptime(stop_time[i], "%m/%d/%Y %H:%M:%S")
    stop_hour.append(trip_stop_time.tm_hour)

In [18]:
# organize useful information to a new dataframe
new_data = pd.DataFrame({
        "month": month,
        "day_of_month": day_of_month,
        "day_of_week": day_of_week,
        "start_hour": start_hour,
        "stop_hour": stop_hour,        
    })

new_data = pd.concat([new_data, main_data['tripduration'], main_data['start_neighborhood'], 
                      main_data['end_neighborhood'], main_data['usertype']], axis = 1)

new_data.head()

,day_of_month,day_of_week,month,start_hour,stop_hour,tripduration,start_neighborhood,end_neighborhood,usertype
0,1,4,7,0,0,191,Greenpoint,Greenpoint,Subscriber
1,1,4,7,0,0,687,Chelsea and Clinton,Greenwich Village and Soho,Customer
2,1,4,7,0,0,609,Greenwich Village and Soho,Chelsea and Clinton,Customer
3,1,4,7,0,0,672,Chelsea and Clinton,Greenwich Village and Soho,Subscriber
4,1,4,7,0,0,466,Greenwich Village and Soho,Greenwich Village and Soho,Subscriber


In [19]:
new_data.to_csv("new_trip_data.csv", sep = ',')

### Compute data for each region ###

In [27]:
data_organized = []
# create labels
labels = ["month", "day_of_month", "day_of_week", "neighborhood", "hour", "trip_number", 
          "average_duration", "percent_subscriber", "trip_type"]
data_organized = np.asarray(labels).reshape(1, 9)

for month_num in new_data['month'].unique():
    # operate on data of this month
    month_data = new_data.loc[new_data['month'] == month_num]
    for day_num in month_data['day_of_month'].unique():
        # operate on data of each day
        day_data = month_data.loc[month_data['day_of_month'] == day_num]
        weekday = day_data['day_of_week'].values[0]
        # outgoing trips in each region
        for region in day_data['start_neighborhood'].unique():
            region_data = day_data.loc[day_data['start_neighborhood'] == region]
            # compute for each hour
            for hour in region_data['start_hour'].unique():
                # prepare information of this piece of data
                summary = []
                summary.append(month_num)
                summary.append(day_num)
                summary.append(weekday)
                summary.append(region)
                summary.append(hour)
                # target dataset
                final_data = region_data.loc[region_data['start_hour'] == hour]
                # compute number of outgoing trips
                summary.append(final_data.shape[0])
                # average trip duration
                summary.append(np.mean(final_data['tripduration'].values))
                # compute subscriber percentage
                percent = np.mean(final_data['usertype'] == "Subscriber")
                summary.append(percent)
                # label trip type
                summary.append("outgoing")
                
                # append the list to a large array
                summary = np.asarray(summary).reshape((1, 9))
                data_organized = np.concatenate((data_organized, summary), axis = 0)

        # incoming trips in each region   
        for region in day_data['end_neighborhood'].unique():
            region_data = day_data.loc[day_data['end_neighborhood'] == region]
            # compute for each hour
            for hour in region_data['stop_hour'].unique():
                # prepare information of this piece of data
                summary = []
                summary.append(month_num)
                summary.append(day_num)
                summary.append(weekday)
                summary.append(region)
                summary.append(hour)
                # target dataset
                final_data = region_data.loc[region_data['stop_hour'] == hour]
                # compute number of incoming trips
                summary.append(final_data.shape[0])
                # average trip duration
                summary.append(np.mean(final_data['tripduration'].values))
                # compute subscriber percentage
                percent = np.mean(final_data['usertype'] == "Subscriber")
                summary.append(percent)
                # label trip type
                summary.append("incoming")
                
                # append the list to a large array
                summary = np.asarray(summary).reshape((1, 9))
                data_organized = np.concatenate((data_organized, summary), axis = 0)

In [28]:
print "Dimension: ", data_organized.shape 

Dimension:  (57019L, 9L)


In [31]:
df_data_organized = pd.DataFrame(data_organized[1:, :], columns= data_organized[0, :])
df_data_organized.head()

,month,day_of_month,day_of_week,neighborhood,hour,trip_number,average_duration,percent_subscriber,trip_type
0,7,1,4,Greenpoint,0,26,1888.5,0.961538461538,outgoing
1,7,1,4,Greenpoint,1,12,524.25,1.0,outgoing
2,7,1,4,Greenpoint,2,9,852.222222222,1.0,outgoing
3,7,1,4,Greenpoint,3,4,563.25,1.0,outgoing
4,7,1,4,Greenpoint,4,3,470.333333333,1.0,outgoing


### Total docks in each neighborhood region ###

In [33]:
station_data = pd.read_csv("station_info.csv", delimiter=",", low_memory=False)
print "Dimension of station data: ", station_data.shape
station_data.head()

Dimension of station data:  (663, 19)


,Unnamed: 0,altitude,availableBikes,availableDocks,city,id,landMark,lastCommunicationTime,latitude,location,longitude,postalCode,stAddress1,stAddress2,stationName,statusKey,statusValue,testStation,totalDocks
0,0,NaN,18,20,NaN,72,NaN,2016-11-22 02:41:46 PM,40.767272,NaN,-73.993929,NaN,W 52 St & 11 Ave,NaN,W 52 St & 11 Ave,1,In Service,False,39
1,1,NaN,29,4,NaN,79,NaN,2016-11-22 02:41:23 PM,40.719116,NaN,-74.006667,NaN,Franklin St & W Broadway,NaN,Franklin St & W Broadway,1,In Service,False,33
2,2,NaN,22,5,NaN,82,NaN,2016-11-22 02:42:36 PM,40.711174,NaN,-74.000165,NaN,St James Pl & Pearl St,NaN,St James Pl & Pearl St,1,In Service,False,27
3,3,NaN,18,43,NaN,83,NaN,2016-11-22 02:44:54 PM,40.683826,NaN,-73.976323,NaN,Atlantic Ave & Fort Greene Pl,NaN,Atlantic Ave & Fort Greene Pl,1,In Service,False,62
4,4,NaN,28,9,NaN,116,NaN,2016-11-22 02:44:42 PM,40.741776,NaN,-74.001497,NaN,W 17 St & 8 Ave,NaN,W 17 St & 8 Ave,1,In Service,False,39


In [34]:
station_in_service = station_data.loc[station_data['statusKey'] == 1]
print "Dimension of station in-service: ", station_in_service.shape

Dimension of station in-service:  (638, 19)


In [43]:
# match the stations to neighborhood
n_station =  station_in_service.shape[0]
station_neighborhood = []

# 0 if using the start_neighborhood, 1 if using the end_neighborhood
end_station = 0

for n in range(n_station):
    station_id = station_in_service['id'].values[n]
    same_station = main_data.loc[main_data['start.station.id'] == station_id]
    if same_station.shape[0] == 0:
        same_station = main_data.loc[main_data['end.station.id'] == station_id]
        end_station = 1
    if same_station.shape[0] == 0:
        region = 'NA'
    elif end_station == 0:
        region = same_station['start_neighborhood'].values[0]
    else:
        region = same_station['end_neighborhood'].values[0]
    end_station = 0
    station_neighborhood.append(region)
    
print "Check dimension of the matched neighborhood: ", len(station_neighborhood)

 Check dimension of the matched neighborhood:  638


**Check the stations without matched neighborhood. It turns out that the stations are all new ones (with large station id number) that probably did not operation before Sep 2016. **

In [57]:
station_neighborhood = np.asarray(station_neighborhood)

station_not_in_service = station_neighborhood[station_neighborhood == "NA"]

print "Number of stations not-in-service: ", len(station_not_in_service)

Number of stations not-in-service:  41


In [58]:
station_in_service['id'].values[station_neighborhood == "NA"]

array([3187, 3188, 3189, 3190, 3191, 3192, 3193, 3194, 3195, 3196, 3197,
       3200, 3201, 3202, 3203, 3205, 3206, 3207, 3209, 3210, 3211, 3212,
       3215, 3216, 3217, 3220, 3268, 3269, 3270, 3271, 3272, 3273, 3274,
       3275, 3277, 3278, 3280, 3281, 3346, 3407, 3438], dtype=int64)

**Compute total number of docks in each region, and add the information to the organized data set. **

In [60]:
station_in_service['neighborhood'] = pd.Series(station_neighborhood)

D:\Anaconda2\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [62]:
# check how many regions there are
station_in_service['neighborhood'].unique()

array(['Chelsea and Clinton', 'Greenwich Village and Soho',
       'Lower Manhattan', 'Northwest Brooklyn', 'Central Brooklyn',
       'Lower East Side', 'Gramercy Park and Murray Hill',
       'Bushwick and Williamsburg', 'Upper West Side', 'Greenpoint',
       'Northwest Queens', 'Upper East Side', 'Hudson', 'NA',
       'East Harlem', 'Central Harlem', 'Canarsie and Flatlands', nan], dtype=object)

In [67]:
# compute total docks in each region
docks_by_region = ["neighborbood", "totalDocks"]
docks_by_region = np.array(docks_by_region).reshape((1,2))

for region in station_in_service['neighborhood'].unique():
    region_data = station_in_service.loc[station_in_service['neighborhood'] == region]
    result = [region, np.sum(region_data['totalDocks'].values)]
    result = np.asarray(result).reshape((1, 2))
    
    docks_by_region = np.concatenate((docks_by_region, result), axis = 0)

In [68]:
docks_by_region

array([['neighborbood', 'totalDocks'],
       ['Chelsea and Clinton', '2435'],
       ['Greenwich Village and Soho', '1168'],
       ['Lower Manhattan', '1039'],
       ['Northwest Brooklyn', '3691'],
       ['Central Brooklyn', '759'],
       ['Lower East Side', '2067'],
       ['Gramercy Park and Murray Hill', '1470'],
       ['Bushwick and Williamsburg', '1007'],
       ['Upper West Side', '1141'],
       ['Greenpoint', '964'],
       ['Northwest Queens', '381'],
       ['Upper East Side', '1197'],
       ['Hudson', '397'],
       ['NA', '1173'],
       ['East Harlem', '374'],
       ['Central Harlem', '39'],
       ['Canarsie and Flatlands', '27'],
       ['nan', '0.0']], 
      dtype='|S32')

In [83]:
# normalize the trip numbers by total docks in the region
n_regions = docks_by_region.shape[0] - 1

# extract information from the organized data
neighborhood = data_organized[1:,3]
trip_number = data_organized[1:,5].astype(int)
# array to store the normalized trip number
normalized_trip_num = np.zeros(len(trip_number))

for n in range(n_regions):
    region = docks_by_region[n+1, 0]
    normalized_trip_num[neighborhood == region] = trip_number[neighborhood == region]/float(docks_by_region[n+1, 1])

In [89]:
# add to the dataframe
df_data_organized['normalized_trip_num'] = normalized_trip_num

df_data_organized.head()

,month,day_of_month,day_of_week,neighborhood,hour,trip_number,average_duration,percent_subscriber,trip_type,normalized_trip_num
0,7,1,4,Greenpoint,0,26,1888.5,0.961538461538,outgoing,0.026971
1,7,1,4,Greenpoint,1,12,524.25,1.0,outgoing,0.012448
2,7,1,4,Greenpoint,2,9,852.222222222,1.0,outgoing,0.009336
3,7,1,4,Greenpoint,3,4,563.25,1.0,outgoing,0.004149
4,7,1,4,Greenpoint,4,3,470.333333333,1.0,outgoing,0.003112


### Label time by groups ###

In [109]:
# function to convert hours into groups
# input: hours 0-24
# output: new group

def match(input_hour):
    if input_hour < 6 or input_hour > 21:
        output_hour = 'night'
    elif input_hour < 8:
        output_hour = '7'
    elif input_hour < 9:
        output_hour = '8'
    elif input_hour < 10:
        output_hour = '9'
    elif input_hour < 17:
        output_hour = '10-16'
    elif input_hour < 18:
        output_hour = '17'
    elif input_hour < 19:
        output_hour = '18'
    elif input_hour < 20:
        output_hour = '19'
    elif input_hour < 21:
        output_hour = '20'
    else:
        output_hour = '21'
    
    return (output_hour)

In [110]:
hours = df_data_organized['hour'].values.astype(int)
hour_group = []

for n in range(len(hours)):
    group = match(hours[n])
    hour_group.append(group)
    
hour_group = np.asarray(hour_group).reshape((len(hours), 1))

# combined with the organized data
df_data_organized['hour_group'] = hour_group

In [111]:
# save the data
df_data_organized.to_csv("organized_data.csv", sep = ',')

### Add normalized imbalance ###
normalized number of outgoing trips - normalized number of incoming trips 

In [123]:
data_organized_2 = []
# create labels
labels = ["month", "day_of_month", "day_of_week", "neighborhood", "hour", "normalized_imbalance"]
data_organized_2 = np.asarray(labels).reshape(1, 6)

for month_num in df_data_organized['month'].unique():
    # operate on data of this month
    month_data = df_data_organized.loc[df_data_organized['month'] == month_num]
    for day_num in month_data['day_of_month'].unique():
        # operate on data of each day
        day_data = month_data.loc[month_data['day_of_month'] == day_num]
        weekday = day_data['day_of_week'].values[0]
        # for each region
        for region in day_data['neighborhood'].unique():
            region_data = day_data.loc[day_data['neighborhood'] == region]
            # compute for each hour
            for hour in region_data['hour'].unique():
                # prepare information of this piece of data
                summary = []
                summary.append(month_num)
                summary.append(day_num)
                summary.append(weekday)
                summary.append(region)
                summary.append(hour)
                # target dataset
                final_data = region_data.loc[region_data['hour'] == hour]
                
                # compute normalized imbalance
                outgoing_data = final_data.loc[final_data['trip_type'] == 'outgoing']['normalized_trip_num'].values
                if len(outgoing_data) > 0:
                    outgoing = outgoing_data[0]
                else:
                    # set to 0 if no outgoing trips
                    outgoing = 0
                    
                incoming_data = final_data.loc[final_data['trip_type'] == 'incoming']['normalized_trip_num'].values
                if len(incoming_data) > 0:
                    incoming = incoming_data[0]
                else:
                    # set to 0 if no incoming trips
                    incoming = 0
                
                imbalance = outgoing - incoming
                summary.append(imbalance)
                
                # append the list to a large array
                summary = np.asarray(summary).reshape((1, 6))
                data_organized_2 = np.concatenate((data_organized_2, summary), axis = 0)      

In [129]:
# convert to dataframe
df_data_organized_2 = pd.DataFrame(data_organized_2[1:, :], columns= data_organized_2[0, :])

# add hour group
hours = df_data_organized_2['hour'].values.astype(int)
hour_group = []

for n in range(len(hours)):
    group = match(hours[n])
    hour_group.append(group)
    
hour_group = np.asarray(hour_group).reshape((len(hours), 1))

# combined with the organized data
df_data_organized_2['hour_group'] = hour_group

df_data_organized_2.head()

,month,day_of_month,day_of_week,neighborhood,hour,normalized_imbalance,hour_group
0,7,1,4,Greenpoint,0,0.0,night
1,7,1,4,Greenpoint,1,-0.0134854771784,night
2,7,1,4,Greenpoint,2,0.00311203319502,night
3,7,1,4,Greenpoint,3,-0.00311203319502,night
4,7,1,4,Greenpoint,4,-0.0051867219917,night


In [130]:
# save the data
df_data_organized_2.to_csv("organized_data_2.csv", sep = ',')